In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyodbc
import re
from datetime import datetime
# URL base de la pag
base_url = "https://www.freyafitness.com.ar/productos/?mpage=3"


# lista para almacenar los productos y precios
productos_totales = []
precios_totales = []
link_producto=[]

# lista de palabras clave de marca
palabras_clave = {
    1: ['Agebiologique', 'Age Biologique', 'Age'],
    2: ['Athom X'],
    3: ['Growth Supplements', '+Growth'],
    4: ['HTN'],
    5: ['HochSport'],
    6: ['Ena', 'ENA SPORT'],
    7: ['Star Nutrition'],
    8: ['Gentech'],
    9: ['Nutremax'],
    10: ['Body Advance', 'BODY ADVANCE', 'BODY'],
    11: ['Nutrilab'],
    12: ['Hardcore Nutrition', 'Hard Core', 'Hard'],
    13: ['Universal Nutrition'],
    14: ['Gold Nutrition'],
    15: ['Mervicklab'],
    16: ['Ultra tech', 'Ultra Tech', 'Ultratech Nutrition'],
    17: ['Vitatech'],
    18: ['Protein Factory'],
    19: ['Protein Project'],
    20: ['Xtrenght', 'xtrength ', 'Xtrenght Nutrition'],
    21: ['Xing Yu'],
    22: ['Erox'],
    23: ['HARDY'],
    24: ['Xxl'],
    25: ['SPX'],
    26: ['Desconocido']
}


# diccionario de categorias y palabras clave
tipos_de_productos = {
    1: ['creatina', 'monohidrato'],
    2: ['proteina', 'Proteínas', 'whey', 'protein', 'Whey'],
    3: ['vitamina', 'multivitaminico'],
    4: ['Collagen', 'Colageno'],
    5: ['Gluta', 'l-glutamina', 'Glutamina'],
    6: ['Quemador De Grasa', 'Carntina', 'Quemador De Grasas'],
    7: ['Aminoácidos', 'Aminoacidos', 'bcaa'],
    8: ['Cafeína', 'Cafeina'],
    9: ['Zma', 'zma', 'magnesio'],
    10: ['Pre Work', 'Pre entreno', 'prework', 'preentreno'],
    11: ['Ganador De Peso', 'Ganador', 'ganador', 'Ultra Mass', 'Mutant Mass'],
    12: ['Bebida Isotónica', 'bebida', 'Bebida'],
    13: ['Gel', 'Energy', 'energy', 'Enargy', 'Race'],
    14: ['Hmb', 'hmb'],
    15: ['Vigorizante', 'vigorizante', 'Potenciador', 'potenciador', 'Testo', 'testo'],
    16:['Desconocido']
}


# obtener la fecha y hora de scraping
fecha_scraping = datetime.now()

# recorrer las paginas 
for pagina in range(1, 27):  
    # URL de la pagina actual
    url = base_url + f"?page={pagina}"
    
    # obtener el HTML de la página actual
    url_obtenido = requests.get(url)
    html_obtenido = url_obtenido.text
    
    # parsear el contenido
    soup = BeautifulSoup(html_obtenido, "html.parser")
    
    # encontrar los productos y precios de la pag actual
    separadores = soup.find_all('a', class_="js-item-name item-name")
    price = soup.find_all('span', class_="js-price-display item-price h6")
    
    # agregar los productos y precios a las listas totales
    for producto, precio in zip(separadores, price):
        productos_totales.append(producto.get('aria-label'))
        precios_totales.append(precio.text.strip())

      #encontrar los links del producto
    links = soup.find_all('a', class_="js-item-name item-name")
    for i in links: 
        link_producto.append(i.get('href'))
df = pd.DataFrame({'Producto': productos_totales, 'Precio': precios_totales, 'Link_Producto': link_producto})

def encontrar_marca(row):
    producto = row['Producto']
    for id_marca, palabras in palabras_clave.items():
        for palabra in palabras:
            if palabra.lower() in producto.lower():
                return id_marca
    return 26  

# funcion para determinar el tipo de producto
def clasificar_tipo_producto(row):
    producto = row['Producto']  
    for tipo, palabras_clave in tipos_de_productos.items():
        for palabra in palabras_clave:
            if re.search(r'\b' + palabra + r'\b', producto, re.IGNORECASE):
                return tipo
    return 16

# agregar la columna "fecha_de_scraping"
df['fecha_scraping'] = fecha_scraping

# agregar la columna para el tipo de producto
df['id_tipo'] = df.apply(clasificar_tipo_producto, axis=1)


# aplicar la funcion a cada fila y crear la columna "marca"
df['id_marca'] = df.apply(encontrar_marca, axis=1)
# Reorganizar el orden de las columnas en el DataFrame
df = df[['Producto', 'id_marca', 'id_tipo', 'Precio', 'fecha_scraping', 'Link_Producto']]


# configurar la conexión a SQL Server
server = ''
database = ''
username = ''
password = ''
driver = '{ODBC Driver 17 for SQL Server}'  

# crear la cadena de conexion
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};'

#conector 
conn = pyodbc.connect(conn_str)

# nombre de la tabla en SQL Server
tabla_sql = 'offsuple'

# convertir el df en una lista de tuplas
records = [tuple(row) for row in df.values]

# query SQL para insertar datos en la tabla
query = f"INSERT INTO {tabla_sql} ({', '.join(df.columns)}) VALUES ({', '.join(['?'] * len(df.columns))})"

# ejecutar la query para insertar los datos
cursor = conn.cursor()
cursor.executemany(query, records)
conn.commit()

# cerrar la conexion
conn.close()

# opciones csv y excel
# df.to_csv('Lista_de_precios_TMS.csv', index=False)
# df.to_excel('Lista_de_precios_TMS.xlsx', index=False)

print("Datos scrapeados y cargados exitosamente en SQL Server.")
